# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

## Clean and Prepare the Data

** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [12]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

## Feature Transformations

In [14]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [16]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [21]:
from pyspark.ml import Pipeline

In [22]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [23]:
cleaner = data_prep_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [26]:
clean_data = clean_data.select(['label','features'])

In [27]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [28]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [29]:
spam_predictor = nb.fit(training)

In [30]:
data.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [31]:
test_results = spam_predictor.transform(testing)

In [32]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,7,15,...|[-658.02198368746...|[1.0,5.0065086058...| 0.0|
 0.0|(13424,[0,1,11,32...|[-866.47684627689...|[1.0,9.3901353555...| 0.0|
 0.0|(13424,[0,1,14,78...|[-688.08191840124...|[1.0,3.8492376456...| 0.0|
 0.0|(13424,[0,1,17,19...|[-804.40237883231...|[1.0,1.3265222793...| 0.0|
 0.0|(13424,[0,1,21,27...|[-1011.2975608142...|[1.0,2.4502233871...| 0.0|
 0.0|(13424,[0,1,24,31...|[-355.29729778159...|[1.0,1.6020870505...| 0.0|
 0.0|(13424,[0,1,27,88...|[-1536.8289456102...|[0.99999999991351...| 0.0|
 0.0|(13424,[0,1,30,12...|[-618.08025439243...|[1.0,1.1830269734...| 0.0|
 0.0|(13424,[0,1,31,43...|[-341.13216360380...|[1.0,5.9281529867...| 0.0|
 0.0|(13424,[0,1,46,17...|[-1135.9620638284...|[3.26940838206142...| 1.0|
 0.0|(13424,[0,1,72,10...|[-701.56383644467...|[1.0,1.6283633861...| 0.0|
 0.0|(13424,[0,1,874,1...|[-97.818132476965...|[0.99999998005395...| 0.0|
 0.0|(13424,[0,1,3657,...|[-127.95824752874...|[0.99998017303752...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-995.24175948333...|[1.0,9.0365293145...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2748.1360462041...|[1.0,4.5610068721...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2748.1360462041...|[1.0,4.5610068721...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-551.71615339696...|[1.0,2.0471017899...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1120.5608060720...|[1.0,8.5177977628...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-471.20369764029...|[0.99999998455558...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-1404.4146381019...|[1.0,5.0252436365...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9221021871206998

### Logistic regression

In [36]:
from pyspark.ml.classification import LogisticRegression

In [37]:
lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(training)
test_results_lr = lr_model.transform(testing)
test_results_lr.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,7,15,...|[36.3819443107501...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,11,32...|[40.6974941464485...|[1.0,2.1149634424...| 0.0|
 0.0|(13424,[0,1,14,78...|[27.5390775615070...|[0.99999999999890...| 0.0|
 0.0|(13424,[0,1,17,19...|[26.7958917113181...|[0.99999999999769...| 0.0|
 0.0|(13424,[0,1,21,27...|[35.0354862081043...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,24,31...|[28.4103572768957...|[0.99999999999954...| 0.0|
 0.0|(13424,[0,1,27,88...|[21.6937423204602...|[0.99999999962109...| 0.0|
 0.0|(13424,[0,1,30,12...|[22.5166573159498...|[0.99999999983360...| 0.0|
 0.0|(13424,[0,1,31,43...|[35.9938937544456...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,46,17...|[15.2561539277933...|[0.99999976322468...| 0.0|
 0.0|(13424,[0,1,72,10...|[23.0479401348815...|[0.99999999990218...| 0.0|
 0.0|(13424,[0,1,874,1...|[17.4020487428545...|[0.99999997230596...| 0.0|
 0.0|(13424,[0,1,3657,...|[15.7069914752929...|[0.99999984915177...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[64.1951159730954...|[1.0,1.3195181482...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[37.0197835276772...|[1.0,8.3658927502...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[37.0197835276772...|[1.0,8.3658927502...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[32.9485556515061...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,2,4,11,...|[46.1312040265689...|[1.0,9.2357601886...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[12.8439396434952...|[0.99999735791527...| 0.0|
 0.0|(13424,[0,2,7,11,...|[23.3909709254746...|[0.99999999993058...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

lr model evaluation

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='label')

In [41]:
AUC = my_eval.evaluate(test_results_lr)
AUC

Out[34]: 0.8246587030716723

In [42]:
acc_lr = acc_eval.evaluate(test_results_lr)
print("Accuracy of lr model at predicting spam was: {}".format(acc_lr))

Accuracy of lr model at predicting spam was: 0.949461822433979

### Random Forest

In [44]:
from pyspark.ml.classification import RandomForestClassifier

In [45]:
rfc = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=200)
rfc_model = rfc.fit(training)
test_results_rfc = rfc_model.transform(testing)
test_results_rfc.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,7,15,...|[173.071398793946...|[0.86535699396973...| 0.0|
 0.0|(13424,[0,1,11,32...|[170.332505747139...|[0.85166252873569...| 0.0|
 0.0|(13424,[0,1,14,78...|[173.552077363775...|[0.86776038681887...| 0.0|
 0.0|(13424,[0,1,17,19...|[167.520624724823...|[0.83760312362411...| 0.0|
 0.0|(13424,[0,1,21,27...|[170.720862089125...|[0.85360431044562...| 0.0|
 0.0|(13424,[0,1,24,31...|[173.425569793849...|[0.86712784896924...| 0.0|
 0.0|(13424,[0,1,27,88...|[170.720862089125...|[0.85360431044562...| 0.0|
 0.0|(13424,[0,1,30,12...|[167.164719792530...|[0.83582359896265...| 0.0|
 0.0|(13424,[0,1,31,43...|[173.425569793849...|[0.86712784896924...| 0.0|
 0.0|(13424,[0,1,46,17...|[170.720862089125...|[0.85360431044562...| 0.0|
 0.0|(13424,[0,1,72,10...|[173.425569793849...|[0.86712784896924...| 0.0|
 0.0|(13424,[0,1,874,1...|[173.425569793849...|[0.86712784896924...| 0.0|
 0.0|(13424,[0,1,3657,...|[173.425569793849...|[0.86712784896924...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[173.854714676655...|[0.86927357338327...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[171.597246773883...|[0.85798623386941...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[171.597246773883...|[0.85798623386941...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[175.933095885778...|[0.87966547942889...| 0.0|
 0.0|(13424,[0,2,4,11,...|[175.578924885875...|[0.87789462442937...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[177.062935889579...|[0.88531467944789...| 0.0|
 0.0|(13424,[0,2,7,11,...|[174.984554680456...|[0.87492277340228...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [46]:
acc_rfc = acc_eval.evaluate(test_results_rfc)
print("Accuracy of rfc model at predicting spam was: {}".format(acc_rfc))

Accuracy of rfc model at predicting spam was: 0.8087136734021008

### Gradient boosted tree

In [48]:
from pyspark.ml.classification import GBTClassifier

In [49]:
gbt = GBTClassifier(featuresCol='features', labelCol='label', maxIter=20)
gbt_model = gbt.fit(training)
test_results_gbt = gbt_model.transform(testing)
test_results_gbt.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,7,15,...|[0.81472811571901...|[0.83609513431802...| 0.0|
 0.0|(13424,[0,1,11,32...|[-0.8008626375686...|[0.16774062197682...| 1.0|
 0.0|(13424,[0,1,14,78...|[0.90226080370356...|[0.85869845655006...| 0.0|
 0.0|(13424,[0,1,17,19...|[0.69372544553298...|[0.80018498059007...| 0.0|
 0.0|(13424,[0,1,21,27...|[-0.8797895739953...|[0.14684305637811...| 1.0|
 0.0|(13424,[0,1,24,31...|[1.31349493275913...|[0.93257853102212...| 0.0|
 0.0|(13424,[0,1,27,88...|[-0.7003920941383...|[0.19769170209264...| 1.0|
 0.0|(13424,[0,1,30,12...|[1.26912865560863...|[0.92678065864953...| 0.0|
 0.0|(13424,[0,1,31,43...|[1.31349493275913...|[0.93257853102212...| 0.0|
 0.0|(13424,[0,1,46,17...|[-0.8797895739953...|[0.14684305637811...| 1.0|
 0.0|(13424,[0,1,72,10...|[1.31349493275913...|[0.93257853102212...| 0.0|
 0.0|(13424,[0,1,874,1...|[1.36786883278973...|[0.93910279490120...| 0.0|
 0.0|(13424,[0,1,3657,...|[1.36786883278973...|[0.93910279490120...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[0.89384958459023...|[0.85664495621953...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[1.06819496582625...|[0.89439010142287...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[1.06819496582625...|[0.89439010142287...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[1.46557848289048...|[0.94936532027350...| 0.0|
 0.0|(13424,[0,2,4,11,...|[1.17814516050068...|[0.91343291877154...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[1.46557848289048...|[0.94936532027350...| 0.0|
 0.0|(13424,[0,2,7,11,...|[0.89384958459023...|[0.85664495621953...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [50]:
acc_gbt = acc_eval.evaluate(test_results_gbt)
print("Accuracy of rfc model at predicting spam was: {}".format(acc_gbt))

Accuracy of rfc model at predicting spam was: 0.9512558066672469

Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!